# Notebook
Notebook for prototyping and experimenting. 

### New Simulation Config


In [ ]:

# beam_settings = load_beam_config(config["beam"])

# lenses = [load_lens_config(lc) for lc in config["lenses"]]
# stages = [load_sim_stage_config(sc) for sc in config["stages"]]

# options = generate_simulation_options(config, "log")
# parameters = generate_simulation_parameters(config)


# sim_config = SimulationConfig(
#     beam=beam_settings,
#     lenses=lenses,
#     stages=stages,
#     parameters=parameters,
#     options=options
# )


# print("SIM CONFIG:")
# pprint(sim_config.beam)
# print("-"*50)
# pprint(sim_config.lenses)
# print("-"*50)
# pprint(sim_config.stages)
# print("-"*50)
# pprint(sim_config.parameters)
# print("-"*50)
# pprint(sim_config.options)



## Electric Field Refactor
Restructure the simulation to use electric field propagation rather than lenses



In [ ]:
%load_ext autoreload
%autoreload 2

from star_glass import utils, plotting
from star_glass.beam import generate_beam
from star_glass.Simulation import (generate_beam_simulation_stage, generate_simulation_parameters, 
    generate_sq_freq_arr, propagate_over_distance)
from star_glass import Simulation
from pprint import pprint
from star_glass.Lens import Lens
from scipy import fftpack
from star_glass.structures import SimulationStage, SimulationParameters, SimulationOptions, SimulationResult
from tqdm import tqdm
import numpy as np
import zarr
import matplotlib.pyplot as plt

path = "test"
config = utils.load_config("example/sim_charizard_2d.yaml")
config["log_dir"] = "log"

sim = Simulation.Simulation(config)
sim.run_simulation()

plotting.save_propagation_gif_full(sim.options.log_dir)


In [ ]:
from IPython.display import Image

plt.close()
fig = plotting.plot_sim_propagation_v2(sim.options.log_dir)
plt.show()

arr = plotting.load_full_sim_propagation_v2(sim.options.log_dir)
fig = plt.figure(figsize=(15, 15))
plt.imshow(np.flip(arr[-4,:, :]), aspect="auto", cmap="turbo")
plt.show()

Image(os.path.join(sim.options.log_dir, "propagation.gif"))



In [ ]:
from star_glass.beam import create_gaussian

path = "test"
config = utils.load_config("example/sim_gaussian_2d.yaml")
config["log_dir"] = path

parameters = generate_simulation_parameters(config)
beam_stage = generate_beam_simulation_stage(config, parameters)

z0 = 400.e-3
r0 = (0, 0)
w0 = 40.e-3

# parameters.sim_wavelength = 30e-3
arr = create_gaussian(r0, w0, z0, parameters=parameters, theta=0, phi=0)

print(parameters)

plt.imshow(np.abs(arr), cmap="turbo", aspect="auto")
plt.title("Gaussian")
plt.colorbar()
plt.show()

np.save("custom/gaussian.npy", arr)

fname = config["beam"]["data"]
loaded = utils.load_np_arr(fname)

plt.imshow(np.abs(loaded), cmap="turbo", aspect="auto")
plt.title("Gaussian (Loaded")
plt.colorbar()
plt.show()


In [ ]:
from PIL import Image
# star https://www.freeiconspng.com/images/white-star-icon
#fair use charizard https://en.wikipedia.org/wiki/Charizard#/media/File:Pok%C3%A9mon_Charizard_art.png
img = Image.open("custom/charizard.png")
img = img.resize((501, 501)).convert("L")
arr = np.asarray(img)
arr = arr>0 * 10000

arr = np.pad(arr, (1000, 1000), mode="constant", constant_values=0)

print(arr.shape)
plt.imshow(arr)
plt.show()

np.save("custom/charizard.npy", arr)

fname = config["beam"]["data"]
loaded = utils.load_np_arr("custom/charizard.npy")

plt.imshow(np.abs(loaded), cmap="turbo", aspect="auto")
plt.title("Charizard (Loaded")
plt.colorbar()
plt.show()

from scipy import fftpack
fft_charizard = fftpack.fft2(arr)

plt.imshow(abs(fft_charizard)**2, cmap="turbo")
plt.show()

np.save("custom/fft_charizard.npy", fft_charizard)


### Vectorisation Upgrade
Convert the propagate wavefront function use vectorised dask calculation. Measure performance

In [ ]:
%load_ext autoreload
%autoreload 2

from star_glass import utils, plotting
from star_glass.Medium import Medium
from star_glass.Simulation import Simulation, calculate_stage_phase, calculate_wavefront_v2, generate_sq_freq_arr, propagate_wavefront_v2
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np

import os

# vector version

# TODO: dask
# notes: slightly faster ... :(




def run_sim_test(vector):
    config = utils.load_config("example/sim_focus_2d.yaml")
    config["log_dir"] = "test"

    sim = Simulation(config)
    options = sim.options
    parameters = sim.parameters

    for stage in sim.sim_stages:

        if stage.wavefront is not None:
            propagation = stage.wavefront

        previous_wavefront = propagation

        # calculate stage phase profile
        phase = calculate_stage_phase(stage, parameters)

        # electric field (wavefront)
        amplitude: float = parameters.A if stage._id == 0 else 1.0
        wavefront = calculate_wavefront_v2(
            phase=phase,
            previous_wavefront=previous_wavefront,
            A=amplitude,
            aperture=stage.lens.aperture,
        )

        ## propagate wavefront
        if vector:
            result = propagate_wavefront_v3(wavefront=wavefront, 
                                stage=stage, 
                                parameters=parameters, 
                                options=options)
        else:
            result = propagate_wavefront_v2(wavefront=wavefront, 
                    stage=stage, 
                    parameters=parameters, 
                    options=options)
        
        propagation = result.propagation

    utils.save_metadata(config, options.log_dir)





In [ ]:
from star_glass import utils, plotting
import napari
import numpy as np
path = r"C:\Users\pcle0002\Documents\repos\star_glass\log\duly-humble-sponge/saving-shiner"
# path = r"C:\Users\pcle0002\Documents\repos\star_glass\log\gently-stable-flea\intent-ibex"
sim = plotting.load_full_sim_propagation_v3(path)


viewer = napari.Viewer(ndisplay=3)
# mask = sim > 1e8
viewer.add_image(sim, name="simulation", scale=[1, 0.1, 0.1] ,rendering="mip", depiction="volume", interpolation="nearest", colormap="turbo")
# viewer.add_image(np.log(sim +1e-12), name="log_simulation", scale=[1, 0.1, 0.1] ,rendering="mip", depiction="volume", interpolation="nearest", colormap="turbo")

In [ ]:
fig = plotting.plot_sim_propagation_v2(path, log=True)
plt.show()

In [ ]:



config = utils.load_config("config.yaml")

from star_glass.Lens  import generate_lens
from star_glass.Medium import Medium

lens = generate_lens(config["lenses"][0],  Medium(2.348), 1e-6)

plotting.plot_lens_profile_2D(lens)
# plt.show()
arr3d = plotting.create_3d_lens(lens)

plotting.view_lens3d_in_napari(arr3d)

In [1]:
%load_ext autoreload
%autoreload 2

### Napari Lens Creator
from napari.layers import Image
import napari.types 
import numpy as np
from magicgui import magicgui
import napari

from pprint import pprint
from star_glass import utils, plotting
from star_glass.Lens  import generate_lens
from star_glass.Medium import Medium
from pathlib import Path

In [ ]:


MICRON_TO_METRE = 1e-6
METRE_TO_MICRON = 1e6

config = utils.load_config("config.yaml")
lens_config = config["lenses"][0]

# core
exponent = lens_config["exponent"]
diameter = lens_config["diameter"] * METRE_TO_MICRON
height = lens_config["height"] * METRE_TO_MICRON
invert = lens_config["inverted"]

# grating
grating = True if lens_config["grating"] is not None else False
grating_width = lens_config["grating"]["width"] * METRE_TO_MICRON
grating_distance = lens_config["grating"]["distance"] * METRE_TO_MICRON
grating_depth = lens_config["grating"]["depth"] * METRE_TO_MICRON
grating_x = lens_config["grating"]["x"]
grating_y = lens_config["grating"]["y"]
grating_centred = lens_config["grating"]["centred"]


# truncation
truncation = True if lens_config["truncation"] is not None else False
truncation_height =  lens_config["truncation"]["height"] * METRE_TO_MICRON
truncation_radius = lens_config["truncation"]["radius"] * METRE_TO_MICRON
truncation_type = lens_config["truncation"]["type"] 
truncation_aperture = lens_config["truncation"]["aperture"]

# aperture
aperture =  True if lens_config["aperture"] is not None else False
aperture_inner = lens_config["aperture"]["inner"] * METRE_TO_MICRON
aperture_outer = lens_config["aperture"]["outer"] * METRE_TO_MICRON
aperture_type = lens_config["aperture"]["type"] 
aperture_invert = lens_config["aperture"]["invert"] 


@magicgui(auto_call=True, call_button="Create Lens", 
  truncation_type={"choices": ["value", "radial"]}, 
  aperture_type={"choices": ["radial", "square"]},
  save_path={"label": "Select a save path:"},
  load_path={"label": "Select a load path:"})
def create_lens(
    load_path: Path = "",
    exponent=exponent, diameter = diameter,
    height=height, invert:bool = invert , 
    grating: bool = grating, 
    grating_width = grating_width, 
    grating_distance = grating_distance, 
    grating_depth = grating_depth, 
    grating_x = grating_x, grating_y = grating_y, grating_centred = grating_centred,
    truncation: bool = truncation,
    truncation_height = truncation_height, truncation_radius = truncation_radius, 
    truncation_type = truncation_type, truncation_aperture = truncation_aperture,
    aperture: bool = aperture,
    aperture_inner =  aperture_inner, aperture_outer = aperture_outer, 
    aperture_type = aperture_type, aperture_invert = aperture_invert,
    pixel_size = 1.0,
    
    save_path: Path = "",

    ) -> napari.types.LayerDataTuple:
  
  if load_path == ".":
    load_path = "config.yaml"

  config = utils.load_config(load_path)
  lens_config = config["lenses"][0]

  # core
  lens_config["height"] = height * MICRON_TO_METRE
  lens_config["diameter"] = diameter * MICRON_TO_METRE
  lens_config["exponent"] = exponent
  lens_config["inverted"] = invert

  # grating
  if grating:
    lens_config["grating"] = {}
    lens_config["grating"]["width"] = grating_width * MICRON_TO_METRE
    lens_config["grating"]["distance"] = grating_distance * MICRON_TO_METRE
    lens_config["grating"]["depth"] = grating_depth * MICRON_TO_METRE
    lens_config["grating"]["x"] = grating_x 
    lens_config["grating"]["y"] = grating_y 
    lens_config["grating"]["centred"] = grating_centred 
  else:
    lens_config["grating"] = None
  
  # truncation
  if truncation:
    lens_config["truncation"] = {}
    lens_config["truncation"]["height"] = truncation_height * MICRON_TO_METRE
    lens_config["truncation"]["radius"] = truncation_radius * MICRON_TO_METRE
    lens_config["truncation"]["type"] = truncation_type 
    lens_config["truncation"]["aperture"] = truncation_aperture
  else:
    lens_config["truncation"] = None
  
  # aperture
  if aperture:
      lens_config["aperture"] = {}
      lens_config["aperture"]["inner"] = aperture_inner * MICRON_TO_METRE
      lens_config["aperture"]["outer"] = aperture_outer * MICRON_TO_METRE
      lens_config["aperture"]["type"] = aperture_type 
      lens_config["aperture"]["invert"] = aperture_invert
  else:
    lens_config["aperture"] = None

  lens = generate_lens(lens_config, Medium(2.348), pixel_size*MICRON_TO_METRE)

  arr3d = plotting.create_3d_lens(lens)

  return [(arr3d, {"name": "Lens", "colormap": "gray", "rendering": "iso", "depiction": "volume"}),
          (lens.aperture, {"name": "Lens Aperture", "opacity": 0.4, "colormap": "yellow", "rendering": "translucent"}, "image"),
          (lens.grating_mask, {"name": "Lens Grating Mask", "opacity": 0.4, "colormap": "green", "rendering": "translucent"}, "image"),
          (lens.truncation_mask, {"name": "Lens Truncation Mask", "opacity": 0.4, "colormap": "cyan", "rendering": "translucent"}, "image")]


# TODO: 
# load config
# load custom
# save lens
# name

viewer = napari.Viewer(ndisplay=3)
viewer.window.add_dock_widget(create_lens)
create_lens(load_path="")


In [2]:

@magicgui(auto_call=False, call_button="View Results", 
  load_path={"label": "Select a simulation path:", "mode": "d" },
  threshold={"max": 1e12})
def view_results(load_path: Path, log_plot: bool = False, threshold: float = 0.0) -> napari.types.LayerDataTuple:

  sim = plotting.load_full_sim_propagation_v3(load_path)

  if log_plot:
    sim = np.log(sim + 1e-12)

  mask = sim>threshold
  sim = sim * mask 

  return [
    (sim, {"name": "simulation", "colormap": "turbo", "scale": [1, 0.1, 0.1]}),
    (mask, {"name": "mask","colormap": "turbo", "scale": [1, 0.1, 0.1]})
  ]

viewer = napari.Viewer(ndisplay=3)
viewer.window.add_dock_widget(view_results)

path = r"C:\Users\pcle0002\Documents\repos\star-glass\src\star_glass\example\tutorial_simulation_results/active-shark"
view_results(path)


c:\Users\pcle0002\Miniconda3\envs\lens_sim\lib\site-packages\napari_tools_menu\__init__.py:179: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


[(dask.array<mul, shape=(110, 2001, 2001), dtype=float32, chunksize=(7, 251, 251), chunktype=numpy.ndarray>,
  {'name': 'simulation', 'colormap': 'turbo', 'scale': [1, 0.1, 0.1]}),
 (dask.array<gt, shape=(110, 2001, 2001), dtype=bool, chunksize=(7, 251, 251), chunktype=numpy.ndarray>,
  {'name': 'mask', 'colormap': 'turbo', 'scale': [1, 0.1, 0.1]})]